# Mask R-CNN - Detect on Prostate Dataset


This notebook shows how to use trained Mask R-CNN on prostate dataset for a sigle pic. As for large pathology image, we crop each image to several patches. This notebook is designed for showing the detection result for a single pic. You'd need a GPU, though, because the network backbone is a Resnet101, which would be slow to detect on a CPU.

The code of the Prostate dataset can be found in prostate.py.

## Import Module

In [ ]:
# import module from system lib
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scipy.io

In [ ]:
# import module from maskrcnn repo
from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate

%matplotlib inline

# Specify GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

## Configurations

In [ ]:
# Specify the dir that store the prostate dataset
dataset_dir = os.path.join(os.path.dirname(os.getcwd()), "Data_Pre_Processing/cedars-224")
# We do 5-fold validation, specify which fold to be exclude for the current run
held_out_set = 0
# Featch the mean_pixel based on the training data (data exclude the held_out_set)
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
# Configuration
class DetectionConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5
    DETECTION_NMS_THRESHOLD = 1     
    MEAN_PIXEL = np.array(mean_pixel)
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_DIM = 512
    DETECTION_CROP = [128, 384, 128, 384] # [height_crop_start, height_crop_end, width_crop_start, width_crop_end]
    MODE = 16
    USE_TUMORCLASS = True
detection_config = DetectionConfig()
detection_config.display()

## Create Model Graph and Loading Weights

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="detection", 
                          config=detection_config,
                          model_dir=MODEL_DIR)
# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights
h5_filename = None # Specify the h5 filename here if you want to choose a specific file
epoch = 71

if h5_filename is not None:
    model_path = os.path.join(ROOT_DIR, ".h5 file name here")
elif epoch == -1:    
    model_path = model.find_last()[1]
else:
    try:
        model_path = model.find_specific(epoch = epoch, held_out_set = held_out_set)[1]
    except:
        model_path = model.find_specific(epoch = epoch)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

## Prepare the Dataset and Specify the Interested Image

In [ ]:
Random = True # whether to randomly choose the image
# Specify the image that is interested in
val_list = [0]
# val_list = [image for image in val_list if image not in exclude_list]
image_patch_num = 2 # from 0 to mode_number - 1
if Random:
    _, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)    
    image_id = random.choice(val_list)
    val_list = [image_id]
    image_patch_num = np.random.randint(0, detection_config.MODE)

dataset_val = prostate.ProstateDataset()
dataset_val.load_prostate(dataset_dir, val_list, mode = detection_config.MODE)
dataset_val.prepare()

## Load the Dataset and Ground Truth Annotation

In [ ]:
# Load image and ground truth data
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, detection_config,
                           image_patch_num, use_mini_mask=False)
gt_tumor_class = 1 if (sum(gt_class_id)) else 0
# Convert gt_instance mask to gt_sementic mask
gt_sementic_mask = utils.instance_2_sementic(gt_mask, gt_class_id)
gt_sementic_mask = gt_sementic_mask['ATmask']

## Run Detection

In [ ]:
# Run object detection
results = model.detect([image], verbose=0)
r = results[0]
if np.argmax(r['tumor_probs']) == 1:
    det_sementic_mask = r['sementic_mask']
else:
    det_sementic_mask = np.zeros((image.shape[0], image.shape[1]))

## Crop the Image, Annotation, and Detection

In [ ]:
Crop = True
if Crop:
    hv, wv = utils.create_crop_region(detection_config)
    image = image[hv, wv]
    gt_sementic_mask = gt_sementic_mask[hv, wv]
    det_sementic_mask = det_sementic_mask[hv, wv]

## Visualization

In [ ]:
# Visualization the sementic map
visualize.display_sementic(image, gt_sementic_mask,
                            figsize=(8, 8))
visualize.display_sementic(image, det_sementic_mask, 
                            figsize=(8, 8))
# Print Tumor prob detection and ground truth
print('Tumor prob detection:', r['tumor_probs'], '\t','Tumor prob ground truth:', gt_tumor_class)